In [ ]:
import re, gzip, io
import pandas as pd
from pathlib import Path

NB_DIR = Path().resolve()          # where the notebook runs 
PROJECT_ROOT = NB_DIR.parent       # ../ (parent)
DATA_DIR = PROJECT_ROOT / "data"

map_fp  = DATA_DIR / "GSE120575_patient_ID_single_cells.txt.gz"
expr_fp = DATA_DIR / "GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt.gz"

print("Using data dir:", DATA_DIR)


def read_geo_samples_table_gz(path):
    """
    Parse only the SAMPLES section from a GEO metadata .txt.gz worksheet.
    Skips comment lines after SAMPLES that start with '#"'.
    Returns a DataFrame with the actual header row and rows.
    """
    rows = []
    header = None
    in_samples = False

    with gzip.open(path, "rt", encoding="latin1", errors="replace") as f:
        for raw in f:
            s = raw.rstrip("\n")

            if not in_samples:
                if s.strip().startswith("SAMPLES"):
                    in_samples = True
                continue

            # stop when the section ends (blank line after we've started rows, or next ALLCAPS label) <-- test this
            if not s.strip():
                if header is not None:
                    break
                else:
                    continue
            if re.fullmatch(r"[A-Z ]{5,}", s.strip()):
                break

            # skip descriptive comment lines that start with "#"
            if s.startswith('"#') or s.startswith("#"):
                continue

            if header is None:
                header = s.split("\t")
            else:
                rows.append(s.split("\t"))

    if header is None or not rows:
        raise RuntimeError("Couldn't locate a proper SAMPLES table header/rows in the GEO file.")

    return pd.DataFrame(rows, columns=header)

samples_raw = read_geo_samples_table_gz(map_fp)
print("Raw SAMPLES shape:", samples_raw.shape)
print(samples_raw.columns[:10].tolist())
print(samples_raw.head(3))


Raw SAMPLES shape: (16291, 35)
['Sample name', 'title', 'source name', 'organism', 'characteristics: patinet ID (Pre=baseline; Post= on treatment)', 'characteristics: response', 'characteristics: therapy', 'molecule', 'description', 'processed data file ']
  Sample name       title           source name      organism  \
0    Sample 1  A10_P3_M11  Melanoma single cell  Homo sapiens   
1    Sample 2  A11_P1_M11  Melanoma single cell  Homo sapiens   
2    Sample 3  A11_P3_M11  Melanoma single cell  Homo sapiens   

  characteristics: patinet ID (Pre=baseline; Post= on treatment)  \
0                                             Pre_P1               
1                                             Pre_P1               
2                                             Pre_P1               

  characteristics: response characteristics: therapy molecule description  \
0                 Responder               anti-CTLA4                        
1                 Responder               anti-CTLA4   

In [2]:
# Standardize columns
samples = samples_raw.rename(columns={
    "title": "CellID",
    "characteristics: patinet ID (Pre=baseline; Post= on treatment)": "PatientTimepoint",
    "characteristics: response": "Response",
    "characteristics: therapy": "Therapy",
})

# Keep just the essentials
samples = samples[["CellID", "PatientTimepoint", "Response", "Therapy"]].copy()

# Split "PatientTimepoint" like "Pre_P1" or "Post_P1_2"
def split_patient_time(val):
    import pandas as pd, re
    if pd.isna(val):
        return pd.Series({"Timepoint": pd.NA, "PatientID": pd.NA})
    parts = str(val).split("_")
    timepoint = parts[0] if parts else pd.NA
    pid = next((p for p in parts if re.fullmatch(r"P\d+", p)), pd.NA)
    return pd.Series({"Timepoint": timepoint, "PatientID": pid})

samples[["Timepoint","PatientID"]] = samples["PatientTimepoint"].apply(split_patient_time)
samples.drop(columns=["PatientTimepoint"], inplace=True)

# Normalize labels a bit
samples["Response"] = samples["Response"].str.strip().str.replace("-", " ").str.title()
samples["Therapy"]  = samples["Therapy"].str.strip()

print(samples.head(5))
print("Timepoint counts:\n", samples["Timepoint"].value_counts())
print("Response counts:\n", samples["Response"].value_counts())
print("Unique patients:", samples["PatientID"].nunique())


       CellID   Response     Therapy Timepoint PatientID
0  A10_P3_M11  Responder  anti-CTLA4       Pre        P1
1  A11_P1_M11  Responder  anti-CTLA4       Pre        P1
2  A11_P3_M11  Responder  anti-CTLA4       Pre        P1
3  A11_P4_M11  Responder  anti-CTLA4       Pre        P1
4  A12_P3_M11  Responder  anti-CTLA4       Pre        P1
Timepoint counts:
 Timepoint
Post    10363
Pre      5928
Name: count, dtype: int64
Response counts:
 Response
Non Responder    10727
Responder         5564
Name: count, dtype: int64
Unique patients: 32


In [ ]:

def read_expression_tsv_gz_fix_firstcol(path):
    """
    Reads a very-wide TSV.gz of gene-by-cell expression.
    Some lines may have one extra tab (len(parts) = ncols+1) due to a stray tab in the first field.
    We fix by merging the overflow back into the first column until len(parts)==ncols.
    Returns a DataFrame with first column as index (genes) and remaining columns as cells.
    """
    rows = []
    with gzip.open(path, "rt", encoding="latin1", errors="replace") as f:
        header_line = f.readline().rstrip("\n")
        header = header_line.split("\t")
        ncols = len(header)

        for line in f:
            s = line.rstrip("\n")
            if not s:  # skip blank
                continue
            parts = s.split("\t")

            # If there are too many fields, merge into first column
            while len(parts) > ncols:
                # merge parts[0] and parts[1] with a space (or underscore)
                parts[0] = parts[0] + " " + parts[1]
                del parts[1]

            # If there are too few fields pad with empty
            while len(parts) < ncols:
                parts.append("")

            rows.append(parts)

    df = pd.DataFrame(rows, columns=header)
    # set first column (gene symbol/id) as index
    df = df.set_index(df.columns[0])
    # convert numeric cells; keep non-numeric as NaN
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

expr = read_expression_tsv_gz_fix_firstcol(expr_fp)
print("Expression matrix shape (genes x cells):", expr.shape)
print("First 5 gene index:", expr.index[:5].tolist())
print("First 5 cell columns:", expr.columns[:5].tolist())


Expression matrix shape (genes x cells): (55738, 16291)
First 5 gene index: ['', 'TSPAN6 0.00', 'TNMD 0.00', 'DPM1 0.00', 'SCYL3 0.00']
First 5 cell columns: ['A10_P3_M11', 'A11_P1_M11', 'A11_P3_M11', 'A11_P4_M11', 'A12_P3_M11']


In [4]:

# samples has columns: CellID, PatientID, Timepoint, Response, Therapy
cell_ids_in_meta = samples["CellID"].astype(str).unique()

# intersect with expression columns
common_cells = expr.columns.intersection(cell_ids_in_meta)
print("Cells in expression:", expr.shape[1])
print("Cells with metadata:", len(cell_ids_in_meta))
print("Common cells:", len(common_cells))

# keep only common cells
expr = expr.loc[:, common_cells].copy()
meta = samples.set_index("CellID").loc[common_cells].reset_index()

print("Aligned expression shape:", expr.shape)   # genes x common_cells
print("Aligned metadata shape:", meta.shape)     # common_cells x 5
print(meta.head())


Cells in expression: 16291
Cells with metadata: 16291
Common cells: 16291
Aligned expression shape: (55738, 16291)
Aligned metadata shape: (16291, 5)
        index   Response     Therapy Timepoint PatientID
0  A10_P3_M11  Responder  anti-CTLA4       Pre        P1
1  A11_P1_M11  Responder  anti-CTLA4       Pre        P1
2  A11_P3_M11  Responder  anti-CTLA4       Pre        P1
3  A11_P4_M11  Responder  anti-CTLA4       Pre        P1
4  A12_P3_M11  Responder  anti-CTLA4       Pre        P1


In [7]:
# library size per cell
libs = expr.sum(axis=0)
print(libs.describe())
# What do gene labels look like?
print("Index dtype:", expr.index.dtype)
print(list(expr.index[:10]))

# Quick stats on common formats
n_ensg = sum(idx.startswith("ENSG") for idx in expr.index.astype(str))
n_mt   = sum(str(idx).upper().startswith(("MT-","MT.")) for idx in expr.index.astype(str))
print("Rows starting with ENSG:", n_ensg, "of", len(expr.index))
print("Rows looking like mitochondrial (MT-):", n_mt)


count    16291.000000
mean     15327.147957
std       4742.054091
min          0.000000
25%      12087.750000
50%      14193.820000
75%      17230.520000
max      44286.830000
dtype: float64
Index dtype: object
['', 'TSPAN6 0.00', 'TNMD 0.00', 'DPM1 0.00', 'SCYL3 0.00', 'C1orf112 0.00', 'FGR 0.00', 'CFH 0.00', 'FUCA2 0.00', 'GCLC 0.00']
Rows starting with ENSG: 0 of 55738
Rows looking like mitochondrial (MT-): 37
